# Manual Partitioning Tutorial

In this tutorial we will run through how to manually partition a graph.  There are six steps:

1. Import ResNet50 code from torchvision and set to evaluation mode
1. Download a test image and preprocess it
1. Run inference on CPU as a baseline
1. Manually partition the graph using Neuron
1. Save the model to be loaded on another instance
1. Inspect the graph to deepen our understanding

The following is a ResNet50 implementation copied from `torchvision.models.resnet`.

**STEP 1:** Import torchvision ResNet50 and run the model on CPU

Note that training code can be inserted before `model.eval()` if retraining/fine-tuning is necessary.

In [ ]:
import torch
import torch.nn as nn


def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x


In [ ]:
from torch.utils.model_zoo import load_url as load_state_dict_from_url

model = ResNet(Bottleneck, [3, 4, 6, 3])
state_dict = load_state_dict_from_url('https://download.pytorch.org/models/resnet50-19c8e357.pth', progress=True)
model.load_state_dict(state_dict)
# you can do some training here, before calling model.eval()
model.eval()
print('ResNet50 model is turned into inference mode')

**STEP 2:** Download a cat image and preprocess it

In [ ]:
import os
import numpy as np
from torchvision import transforms, datasets
from tensorflow.keras.applications import resnet50
import urllib.request

imagedir = './images'
os.makedirs(imagedir, exist_ok=True)
urllib.request.urlretrieve(
    'https://raw.githubusercontent.com/awslabs/mxnet-model-server/master/docs/images/kitten_small.jpg',
    os.path.join(imagedir, 'kitten_small.jpg'),
)
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
eval_dataset = datasets.ImageFolder(
    '.',
    transforms.Compose([
        transforms.Resize([224, 224]),
        transforms.ToTensor(),
        normalize,
    ])
)
image, label = eval_dataset[0]
image = torch.tensor(image.numpy()[np.newaxis, ...])

**STEP 3:** Run inference without neuron for comparison

In [ ]:
print('model inference result:')
print(resnet50.decode_predictions(model(image).detach().numpy(), top=5)[0])

STEP 4: Run the same inference using torch.jit.trace - the n we can save and load teh model

In [ ]:
jit_trace = torch.jit.trace(model, example_inputs=image)
print('jit.trace inference result:')
print(resnet50.decode_predictions(jit_trace(image).detach().numpy(), top=5)[0])

In [ ]:
jit_trace_filename = 'resnet50_jit_trace.pt'
jit_trace.save(jit_trace_filename)
jit_trace_loaded = torch.jit.load(jit_trace_filename)
print('loaded jit.trace inferenced result:')
print(resnet50.decode_predictions(jit_trace_loaded(image).detach().numpy(), top=5)[0])

**STEP 4:** Manually partition the ResNet50 model and execute it

To generate a Neuron-optimized TorchScript with only layers 1~4 placed on Neuron runtime, we first define a new module class `ResNetNeuron` inheriting from `ResNet`. We add 'torch.neuron.trace' calls in the forward function of this module in order to turn layer submodules into Neuron-optimized ones.

In [ ]:
import torch.neuron

class ResNetNeuron(ResNet):
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        self.layer1 = torch.neuron.trace(self.layer1, x, fallback=False)
        x = self.layer1(x)
        self.layer2 = torch.neuron.trace(self.layer2, x, fallback=False)
        x = self.layer2(x)
        self.layer3 = torch.neuron.trace(self.layer3, x, fallback=False)
        x = self.layer3(x)
        self.layer4 = torch.neuron.trace(self.layer4, x, fallback=False)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

Now construct the class and runn an inference to trigger the `neuron-cc` compiler.  Watch for the [ \* ] icon to the left of this cell to disappear and show a number - this will take a minute or two to run

In [ ]:
model_neuron = ResNetNeuron(Bottleneck, [3, 4, 6, 3])
model_neuron.load_state_dict(state_dict)
model_neuron.eval()
result = model_neuron(image)  # this line triggers neuron-cc compiler
print('Neuron optimized model inference result:')
print(resnet50.decode_predictions(result.detach().numpy(), top=5)[0])

**STEP 5:** Save the model as torchscript ready to deploy

To deploy the Neuron-optimized as a TorchScript, we use `torch.jit.trace` again to generate torchscript for the entire mode, including the Neuron-optimized `ScriptModule`

In [ ]:
neuron_trace = torch.jit.trace(model_neuron, example_inputs=image)
print('neuron.trace inference result:')
print(resnet50.decode_predictions(neuron_trace(image).detach().numpy(), top=5)[0])

This Neuron-optimized `ScriptModule` can be saved/loaded easily and be deployed on inf1 instances.

In [ ]:
neuron_trace_filename = 'resnet50_neuron_trace.pt'
neuron_trace.save(neuron_trace_filename)
neuron_trace_loaded = torch.jit.load(neuron_trace_filename)
print('loaded neuron.trace inference result:')
print(resnet50.decode_predictions(neuron_trace_loaded(image).detach().numpy(), top=5)[0])

**STEP 6:** Understanding the neuron graph

We can inspect the graph property of the Neuron-optimized `ScriptModule` to get an idea of how Neuron-optimization is performed. Basically, each `torch.neuron.trace` call fuses a submodule (layer) into a `neuron::forward` operator.

In [ ]:
neuron_trace.graph